In [154]:
#youtube url = https://youtu.be/3ocsQiwy5cQ
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [155]:
def nothing(x):
    pass

In [156]:
drawing = False  # 마우스 버튼이 눌렸는지 여부
ix, iy = -1, -1  # 초기 좌표 값
rect_coords = None  # 그린 사각형의 좌표를 저장

# 이미지 로드 및 복사
img = cv2.imread('mountain.jpg')
img_copy = img.copy()
# 마우스 콜백 함수
def draw_circle(event, x, y, flags, param):
    global ix, iy, drawing, img_copy, rect_coords

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

        
    # 마우스가 움직일 때, 사각형을 그리고 내부를 붉게 표시
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()  # 원본 이미지로 복원
            overlay = img.copy()  # 오버레이 이미지 생성
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 0, 240), 2)  # 사각형을 적외선 색상으로
            alpha = 0.3  # 투명도 설정
            cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)  # 적외선 효과 적용
            cv2.putText(img_copy, f'Mouse Position: ({ix}, {iy}) - ({x}, {y})', 
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

    # 마우스 버튼을 뗐을 때, 사각형 내부를 붉게 유지
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rect_coords = (ix, iy, x, y)  # 사각형의 좌표 저장
        overlay = img.copy()  # 오버레이 이미지 생성
        cv2.rectangle(overlay, (ix, iy), (x, y), (0, 0, 240), -1)  # 적외선 색상으로 사각형 내부 채우기
        alpha = 0.3  # 투명도 설정
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)  # 적외선 효과 적용
        cv2.putText(img, f'Mouse Position: ({ix}, {iy}) - ({x}, {y})', 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)




In [157]:
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)

# 트랙바 이름을 "value"로 변경
cv2.createTrackbar('value', 'image', 0, 10, nothing)
while True:
    # 사각형이 그려졌을 때 트랙바 값에 따른 처리
    if rect_coords:
        x1, y1, x2, y2 = rect_coords
        img_display = img.copy()  # 원본 이미지 복사
        # 트랙바 값 가져오기
        m = cv2.getTrackbarPos('value', 'image')
        # 사각형 내부를 선명하게 처리
        if m >= 0:
            sharp_area = cv2.GaussianBlur(img[y1:y2, x1:x2], (0, 0), max(1, 10 - m))  # 선명도 조절
            img_display[y1:y2, x1:x2] = sharp_area
        # 사각형 테두리와 텍스트 유지
        cv2.rectangle(img_display, (x1, y1), (x2, y2), (0, 0, 240), 2)
        cv2.putText(img_display, f'Mouse Position: ({x1}, {y1}) - ({x2}, {y2})', 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.imshow('image', img_display)
    else:
        cv2.imshow('image', img_copy)

    # ESC를 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()